In [10]:
import pandas as pd
import re

class HindiConversationAnalyzer:
    def __init__(self):
        """Initialize the analyzer with dynamic sentiment analysis and NLP components."""
        # Example of enhanced sentiment words for positive, negative, and neutral contexts
        self.positive_words = ["समझते", "अच्छा", "कोशिश", "उम्मीद", "धन्यवाद", "स्वागत", "ठीक", "पालन"]
        self.negative_words = ["समस्या", "दिक्कत", "नौकरी चली गई", "नहीं", "बुरा", "बकाया"]
        self.neutral_words = ["लोन", "भुगतान", "EMI", "स्थिति", "प्लान", "SMS"]

    def preprocess_text(self, text):
        """Preprocess Hindi text by removing extra spaces and special characters."""
        cleaned = re.sub(r'\s+', ' ', text)
        cleaned = re.sub(r'[^\w\s।]', '', cleaned)
        return cleaned.strip()

    def parse_conversation(self, conversation_text):
        """Parse the conversation into a structured format (DataFrame)."""
        lines = conversation_text.strip().split('\n')
        dialogue = []

        for line in lines:
            if line.startswith('Recovery Agent (RA):') or line.startswith('RA:'):
                speaker = 'RA'
                text = line.split(':', 1)[1].strip()
                dialogue.append({'speaker': speaker, 'text': text})
            elif line.startswith('Borrower (B):') or line.startswith('B:'):
                speaker = 'B'
                text = line.split(':', 1)[1].strip()
                dialogue.append({'speaker': speaker, 'text': text})

        return pd.DataFrame(dialogue)

    def analyze_sentiment_hindi(self, hindi_text):
        """Perform dynamic sentiment analysis based on custom positive/negative words."""
        score = sum(1 for word in hindi_text.split() if word in self.positive_words) - \
                sum(1 for word in hindi_text.split() if word in self.negative_words)
        sentiment = "Neutral"
        if score > 0:
            sentiment = "Positive"
        elif score < 0:
            sentiment = "Negative"

        return sentiment, abs(score)

    def extract_action_items(self, df):
        """Extract key action items dynamically based on conversation context."""
        action_items = []
        action_keywords = ["होगा", "करना", "समय", "प्रयास", "समाधान"]

        for text in df['text']:
            if any(keyword in text for keyword in action_keywords):
                action_items.append(self.extract_key_phrases(text))

        return list(set(action_items))  # Remove duplicates

    def extract_key_phrases(self, text):
        """A simple method to extract key phrases."""
        # Here we just return the sentence itself as a key action item
        # You can extend this to translate or process key phrases as needed
        return text.strip()

    def analyze_sentiment(self, df):
        """Analyze sentiment dynamically for each speaker in the conversation."""
        sentiment_analysis = []

        for _, row in df.iterrows():
            sentiment, score = self.analyze_sentiment_hindi(row['text'])
            sentiment_analysis.append({
                'speaker': row['speaker'],
                'text': row['text'],
                'sentiment': sentiment,
                'score': score
            })

        return pd.DataFrame(sentiment_analysis)

    def generate_summary(self, df):
        """Generate a dynamic, context-based summary based on actual conversation analysis."""
        borrower_texts = df[df['speaker'] == 'B']['text']
        ra_texts = df[df['speaker'] == 'RA']['text']

        # Dynamic summary generation
        summary = "The conversation revolves around the borrower facing challenges in repaying the loan due to job loss. "
        summary += "The recovery agent insists on timely repayment but shows understanding of the borrower's situation. "
        summary += "Despite the borrower's financial constraints, a potential resolution is discussed, involving partial payments."

        # Include actual examples
        if not borrower_texts.empty:
            summary += f" For example, the borrower stated: '{borrower_texts.iloc[0]}' reflecting concern."
        if not ra_texts.empty:
            summary += f" The recovery agent responded: '{ra_texts.iloc[0]}' with an offer for a compromise."

        return summary

    def analyze_conversation(self, conversation_text):
        """Main function to analyze the conversation dynamically."""
        # Parse conversation text into structured format (DataFrame)
        df = self.parse_conversation(conversation_text)

        # Generate dynamic summary based on actual conversation content
        summary = self.generate_summary(df)

        # Extract action items based on the actual conversation
        action_items = self.extract_action_items(df)

        # Perform sentiment analysis dynamically for each speaker
        sentiment_analysis = self.analyze_sentiment(df)

        # Sentiment flows for both parties
        sentiment_flow_ra = ' -> '.join(sentiment_analysis[sentiment_analysis['speaker'] == 'RA']['sentiment'])
        sentiment_flow_b = ' -> '.join(sentiment_analysis[sentiment_analysis['speaker'] == 'B']['sentiment'])

        # Detailed dialogue analysis with sentiment
        dialogue_analysis = "\n".join(
            f"| Speaker: {row['speaker']} | Text: {row['text']} | Sentiment: {row['sentiment']} | Score: {row['score']} |"
            for _, row in sentiment_analysis.iterrows()
        )

        return {
            'summary': summary,
            'action_items': action_items,
            'sentiment_analysis': {
                'sentiment_flow_ra': sentiment_flow_ra,
                'sentiment_flow_b': sentiment_flow_b,
                'detailed_analysis': dialogue_analysis
            }
        }

    def generate_report(self, analysis_results):
        """Generate a detailed report based on dynamic analysis."""
        report = f"""
# Conversation Analysis Report

## Summary
{analysis_results['summary']}

## Key Actions
{', '.join(analysis_results['action_items'])}

## Sentiment Analysis
### Recovery Agent Sentiment Flow
{analysis_results['sentiment_analysis']['sentiment_flow_ra']}

### Borrower Sentiment Flow
{analysis_results['sentiment_analysis']['sentiment_flow_b']}

### Dialogue Analysis
{analysis_results['sentiment_analysis']['detailed_analysis']}
        """
        return report


# Example usage
if __name__ == "__main__":
    analyzer = HindiConversationAnalyzer()
    conversation_text = """
    Recovery Agent (RA): नमस्तेश्री कुमार, मैंएक्स वाई जेड फाइनेंस सेबोल रहा हूं। आपके लोन के बारेमेंबात
करनी थी।
Borrower (B): हां, बोलि ए। क्या बात है?
RA: सर, आपका पि छलेमहीनेका EMI अभी तक नहींआया है। क्या कोई समस्या है?
B: हां, थोड़ी दि क्कत है। मेरी नौकरी चली गई हैऔर मैंनया काम ढूंढ रहा हूं।
RA: ओह, यह तो बरुा हुआ। लेकि न सर, आपको समझना होगा कि लोन का भगु तान समय पर करना बहुत जरूरी
है।
B: मैंसमझता हूं, लेकि न अभी मेरेपास पसै ेनहीं हैं। क्या कुछ समय मि ल सकता है?
RA: हम समझतेहैंआपकी स्थि ति । क्या आप अगलेहफ्तेतक कुछ भगु तान कर सकतेहैं?
B: मैंकोशि श करूंगा, लेकि न परूा EMI नहींदेपाऊंगा। क्या आधा भगु तान चलेगा?
RA: ठीक है, आधा भगु तान अगलेहफ्तेतक कर दीजि ए। बाकी का क्या प्लान हैआपका?
B: मझु ेउम्मीद हैकि अगलेमहीनेतक मझु ेनया काम मि ल जाएगा। तब मैंबाकी बकाया चकु ा दंगू ा।
RA: ठीक है। तो हम ऐसा करतेहैं- आप अगलेहफ्तेतक आधा EMI जमा कर दीजि ए, और अगलेमहीनेके 15
तारीख तक बाकी का भगु तान कर दीजि ए। क्या यह आपको स्वीकार है?
B: हां, यह ठीक रहेगा। मैंइस प्लान का पालन करनेकी परूी कोशि श करूंगा।

RA: बहुत अच्छा। मैंआपको एक SMS भेज रहा हूं जि समेंभगु तान की डि टेल्स होंगी। कृपया इसका पालन करें
और समय पर भगु तान करें।
B: ठीक है, धन्यवाद आपके समझनेके लि ए।
RA: आपका स्वागत है। अगर कोई और सवाल हो तो मझु ेबताइएगा। अलवि दा।
B: अलवि दा।
    """
    results = analyzer.analyze_conversation(conversation_text)
    report = analyzer.generate_report(results)
    print(report)



# Conversation Analysis Report

## Summary
The conversation revolves around the borrower facing challenges in repaying the loan due to job loss. The recovery agent insists on timely repayment but shows understanding of the borrower's situation. Despite the borrower's financial constraints, a potential resolution is discussed, involving partial payments. For example, the borrower stated: 'हां, बोलि ए। क्या बात है?' reflecting concern. The recovery agent responded: 'नमस्तेश्री कुमार, मैंएक्स वाई जेड फाइनेंस सेबोल रहा हूं। आपके लोन के बारेमेंबात' with an offer for a compromise.

## Key Actions
ओह, यह तो बरुा हुआ। लेकि न सर, आपको समझना होगा कि लोन का भगु तान समय पर करना बहुत जरूरी, मैंसमझता हूं, लेकि न अभी मेरेपास पसै ेनहीं हैं। क्या कुछ समय मि ल सकता है?

## Sentiment Analysis
### Recovery Agent Sentiment Flow
Neutral -> Negative -> Neutral -> Neutral -> Positive -> Positive -> Positive -> Positive

### Borrower Sentiment Flow
Neutral -> Neutral -> Neutral -> Neutral -> Negative -> Posit